# CE7455 Assignment 1
Peng Hongyi (G2105029E)

## Run the provided code at first

### Load data

In [28]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [29]:
import data 

In [30]:
data_dir = './data/wikitext-2'
corpus = data.Corpus(data_dir)
print(f'Train: {corpus.train.shape}, Val: {corpus.valid.shape}, Test: {corpus.test.shape}')

Train: torch.Size([2088628]), Val: torch.Size([217646]), Test: torch.Size([245569])


In [31]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

In [32]:
BATCH_SIZE = 20
EVAL_BATCH_SIZE = 10
train_data = batchify(corpus.train, BATCH_SIZE)
val_data = batchify(corpus.valid, EVAL_BATCH_SIZE)
test_data = batchify(corpus.test, EVAL_BATCH_SIZE)

In [53]:
N_TOKENS = len(corpus.dictionary)
import model
MODEL = "LSTM"
EMSIZE = 200
N_HID = 200
N_LAYERS = 2
DROPOUT = 0.2
TIED = "store_true"
LR = 20
CLIP_TH = 0.25
model = model.RNNModel(MODEL, N_TOKENS, EMSIZE, N_HID, N_LAYERS, DROPOUT, TIED).to(device)

In [54]:
BPTT = 35
def get_batch(source, i):
    seq_len = min(BPTT, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

In [55]:
def repackage_hidden(h):
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [57]:
from torch import nn
import time
import math
criterion = nn.NLLLoss()
model.train()
total_loss = 0
start_time = time.time()
hidden = model.init_hidden(BATCH_SIZE)
for epoch in range(1):
    for batch, i in enumerate(range(0, train_data.size(0)-1, BPTT)):
        data, targets = get_batch(train_data, i)
        model.zero_grad()
        hidden = repackage_hidden(hidden)
        output, hidden = model(data, hidden)
        loss = criterion(output, targets)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), CLIP_TH)
        for p in model.parameters():
            p.data.add_(p.grad, alpha=-LR)
        total_loss += loss.item()

        if batch % 200 == 0 and batch > 0:
            cur_loss = total_loss / 200
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // BPTT, LR,
                elapsed * 1000 / 200, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()


| epoch   0 |   200/ 2983 batches | lr 20.00 | ms/batch 14.17 | loss  5.85 | ppl   346.21
| epoch   0 |   400/ 2983 batches | lr 20.00 | ms/batch 14.28 | loss  5.73 | ppl   308.45
| epoch   0 |   600/ 2983 batches | lr 20.00 | ms/batch 14.51 | loss  5.54 | ppl   255.88
| epoch   0 |   800/ 2983 batches | lr 20.00 | ms/batch 14.33 | loss  5.49 | ppl   243.37
| epoch   0 |  1000/ 2983 batches | lr 20.00 | ms/batch 14.34 | loss  5.45 | ppl   231.66
| epoch   0 |  1200/ 2983 batches | lr 20.00 | ms/batch 14.08 | loss  5.43 | ppl   228.43
| epoch   0 |  1400/ 2983 batches | lr 20.00 | ms/batch 14.19 | loss  5.47 | ppl   236.80
| epoch   0 |  1600/ 2983 batches | lr 20.00 | ms/batch 14.09 | loss  5.62 | ppl   274.80
| epoch   0 |  1800/ 2983 batches | lr 20.00 | ms/batch 14.03 | loss  5.48 | ppl   238.77
| epoch   0 |  2000/ 2983 batches | lr 20.00 | ms/batch 14.17 | loss  5.47 | ppl   238.45
| epoch   0 |  2200/ 2983 batches | lr 20.00 | ms/batch 14.16 | loss  5.38 | ppl   217.58
| epoch   

### Write my own FNN model

class FNNModel(nn.Module):
    def __init__(self, ):
        super().__init__()
        